In [43]:
# Import libraries
import pandas as pd
import numpy as np

In [44]:
# Import cleaned data (output from colaborator 3)
df = pd.read_csv('../data/03_processed/cleaned_dataframe.csv')
df.head()

,sex,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD,BMI,age,cigsPerDay,glucose,sysBP,totChol,heartRate
0,1,0,0,0,0,0,0,0.277024,0.184211,0.000000,0.104520,0.106383,0.140652,0.363636
1,0,0,0,0,0,0,0,0.319680,0.368421,0.000000,0.101695,0.177305,0.234991,0.515152
2,1,1,0,0,0,0,0,0.237518,0.421053,0.285714,0.084746,0.208038,0.226415,0.313131
3,0,1,0,0,1,0,1,0.316045,0.763158,0.428571,0.177966,0.314421,0.192110,0.212121
4,0,1,0,0,0,0,0,0.183228,0.368421,0.328571,0.127119,0.219858,0.295026,0.414141


In [45]:
# Create a function to aggregate variables by grouping variable using pandas groupby. 

def aggregate_by_group(df, group_var, agg_dict):
    """
    Aggregate variables by a grouping column.

    Parameters:
    df (pd.DataFrame): Input DataFrame
    group_var (str or list[str]): Variable/s (column/s) to group by
    agg_dict (dict): Dictionary of aggregations
                     e.g. {'heartRate': mean, 'visits': sum}

    Returns:
    pd.DataFrame: Aggregated DataFrame
    """
    df_agg = df.groupby(group_var)
    df_agg = df_agg.aggregate(agg_dict)
    df_agg = df_agg.reset_index()

    return df_agg

In [46]:
df.columns

Index(['sex', 'currentSmoker', 'BPMeds', 'prevalentStroke', 'prevalentHyp',
       'diabetes', 'TenYearCHD', 'BMI', 'age', 'cigsPerDay', 'glucose',
       'sysBP', 'totChol', 'heartRate'],
      dtype='object')

In [47]:
# Aggregate variables

# Groupping variables:
group_var = ['sex', 'age', 'currentSmoker', 
             'BPMeds', 'prevalentStroke', 'prevalentHyp', 
             'diabetes', 'TenYearCHD']

# Aggregation function:
agg_dict = {
    "cigsPerDay": "mean",
    "totChol": "mean",
    "sysBP": "mean",
    "BMI": "mean",
    "heartRate": "mean",
    "glucose": "mean"
}

df_agg = aggregate_by_group(df, group_var, agg_dict)

In [48]:
# Export aggregated df (.csv)
df_agg.to_csv('../data/04_aggregated/aggregated_dataframe.csv', index=False)

In [49]:
df_agg.head()

,sex,age,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose
0,0,0.000000,1,0,0,0,0,0,0.214286,0.221269,0.130024,0.346583,0.363636,0.135593
1,0,0.026316,0,0,0,0,0,0,0.000000,0.077187,0.115839,0.104217,0.424242,0.081921
2,0,0.026316,1,0,0,0,0,0,0.142857,0.148370,0.160757,0.173655,0.313131,0.122881
3,0,0.052632,0,0,0,0,0,0,0.000000,0.140938,0.124507,0.193448,0.326599,0.112994
4,0,0.052632,1,0,0,0,0,0,0.265306,0.126930,0.122594,0.154283,0.305916,0.100484


In [50]:
# Implement classification (Logistic regression) to distinguish patients vs. controls. 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 1. Split X and Y:
y = df_agg['TenYearCHD'].astype(int)
X = df_agg.drop(['TenYearCHD'], axis=1) # drop all varribales that are not related to the analysis 

# 2. Classify
lr = LogisticRegression(max_iter=200, class_weight="balanced") # Initialize the model
lr.fit(X, y) ## Fit the model on the observed data.

# 3. Check model metrics:

## See how the model performs.
print ('The accuracy of the model is ' + str(round(lr.score(X, y),2)))

## Confusion matrix
y_pred = lr.predict(X)

cm = confusion_matrix(y, y_pred)
tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix:\n", cm)

## Metrics
precision = precision_score(y, y_pred)
sensitivity = recall_score(y, y_pred) 
specificity = tn / (tn + fp)
f_score = f1_score(y, y_pred)

print("Precision:", round(precision,2))
print("Sensitivity (Recall):", round(sensitivity,2))
print("Specificity:", round(specificity,2))
print("F-score:", round(f_score,2))

The accuracy of the model is 0.59
Confusion Matrix:
 [[231 175]
 [105 166]]
Precision: 0.49
Sensitivity (Recall): 0.61
Specificity: 0.57
F-score: 0.54


/home/aina/miniconda3/envs/test_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
## Let's check which is the best hyperparameter configuration:

import matplotlib.pyplot as plt

iters = list([10, 20, 50, 75, 100, 150, 200, 250, 100, 200, 300, 400, 500, 1000, 1500, 2000, 2500, 3000])
sen = list([])
acc =list([])
for i in iters:
    lr = LogisticRegression(max_iter=i, class_weight="balanced")
    lr.fit(X, y)
    y_pred = lr.predict(X)
    sen.append(recall_score(y, y_pred))
    acc.append(lr.score(X, y))
    
plt.plot(iters,sen) 
plt.plot(iters,acc)   
plt.xlabel('# neighbors')
plt.ylabel('performance')
plt.legend(['sensitivity','accuracy'])
plt.show()

In [52]:
# Lets implement now a Random Forest classification method:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 1. Split X and Y:
y = df_agg['TenYearCHD'].astype(int)
X = df_agg.drop(['TenYearCHD'], axis=1) # drop all varribales that are not related to the analysis 

# 2. Classify (RandomForest)
rf = RandomForestClassifier(n_estimators=400, random_state=42, class_weight="balanced_subsample") # Initialize the model
rf.fit(X, y) ## Fit the model on the observed data.

# 3. Check model metrics:

## See how the model performs.
print ('The accuracy of the model is ' + str(round(rf.score(X, y),2)))

## Confusion matrix
y_pred = rf.predict(X)

cm = confusion_matrix(y, y_pred)
tn, fp, fn, tp = cm.ravel()

print("Confusion Matrix:\n", cm)

## Metrics
precision = precision_score(y, y_pred)
sensitivity = recall_score(y, y_pred) 
specificity = tn / (tn + fp)
f_score = f1_score(y, y_pred)

print("Precision:", round(precision,2))
print("Sensitivity (Recall):", round(sensitivity,2))
print("Specificity:", round(specificity,2))
print("F-score:", round(f_score,2))


The accuracy of the model is 1.0
Confusion Matrix:
 [[406   0]
 [  0 271]]
Precision: 1.0
Sensitivity (Recall): 1.0
Specificity: 1.0
F-score: 1.0


In [53]:
# Let's check which is the best parameter selection:

iters = list([10, 20, 50, 75, 100, 150, 200, 250, 100, 200, 300, 400, 500, 1000, 1500, 2000, 2500, 3000])
sen = list([])
acc =list([])
for i in iters:
    rf = RandomForestClassifier(n_estimators=i, random_state=42, class_weight="balanced_subsample") # Initialize the model
    rf.fit(X, y)
    y_pred = rf.predict(X)
    sen.append(recall_score(y, y_pred))
    acc.append(rf.score(X, y))
    
plt.plot(iters,sen) 
plt.plot(iters,acc)   
plt.xlabel('# neighbors')
plt.ylabel('performance')
plt.legend(['sensitivity','accuracy'])
plt.show()